In [ ]:
tknzr = TweetTokenizer(strip_handles=True, reduce_len=True)
nltk.download('stopwords')
stop_words = set(stopwords.words('english')) 
stemmer = PorterStemmer()
ws.load()
p.set_options(p.OPT.HASHTAG, p.OPT.NUMBER, p.OPT.EMOJI, p.OPT.SMILEY)
def format_tweet(row):
    
    p.set_options(p.OPT.HASHTAG, p.OPT.NUMBER, p.OPT.EMOJI, p.OPT.SMILEY,p.OPT.URL, p.OPT.MENTION )
    parsed_tweet = p.parse(row)
    extra = []
    if parsed_tweet.urls:
        for each in parsed_tweet.urls:
            row = row[:each.start_index]+ ' URL '+row[each.end_index:]
            
    if parsed_tweet.mentions:
        for each in parsed_tweet.mentions:
            row = row[:each.start_index]+ ' MNTN '+row[each.end_index:]
            
    if parsed_tweet.hashtags:
        for each in parsed_tweet.hashtags:
            extra+=ws.segment(each.match)
    #print(row)
            
    if parsed_tweet.numbers:
        for each in parsed_tweet.numbers:
            extra.append('NMBR')
    
    if parsed_tweet.hashtags:
        for each in parsed_tweet.hashtags:
            extra.append('HASH')
            
    if parsed_tweet.emojis:
        for each in parsed_tweet.emojis:
            extra.append('EMOJI')
    
    if parsed_tweet.smileys:
        for each in parsed_tweet.smileys:
            extra.append('SMIL')
    
    p.set_options(p.OPT.URL,p.OPT.MENTION,p.OPT.NUMBER)
    row = p.clean(row)
    tokens = tknzr.tokenize(row)
    tokens_formatted = []
    
    for i in range(len(tokens)):
           if tokens[i] in emoji.UNICODE_EMOJI:
                tokens[i] = emoji.demojize(tokens[i])
    
    for each in tokens:
        if each not in stop_words and len(each)> 0:
            tokens_formatted.append(each)

    tokens_formatted+=extra
    return " ".join(tokens_formatted).lower()

In [ ]:
def format_data(row_data):
    data = []
    i=0
    for row in row_data:
        i+=1
        data.append(format_tweet(row))
    return data

In [ ]:
data = []
classes = []
nltk.download('stopwords')
stop_words = set(stopwords.words('english')) 


with open('/Users/thilinara/IronyKaggle/train_edited.csv', 'r') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    for row in spamreader:
        try:
            classes.append(int(row[1]))
        except:
            continue
        data.append(row[2])
traing_data = format_data(data)
i=0
for cl,d in zip(classes,traing_data):
    i+=1
    print(i," ",cl," : ",d)

In [ ]:
traing_data

In [ ]:
submission_test_set_row = []
with open('/Users/thilinara/IronyKaggle/test_edited.csv', 'r') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    for row in spamreader:
        try:
            row = row[1]
        except Exceptions as ex:
            print(ex)
            continue
        submission_test_set_row.append(row)
submission_test_set_formatted = format_data(submission_test_set_row)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer(ngram_range=(1,3
                                         ), min_df=2)

X_train_counts = count_vect.fit_transform(traing_data)
X_train_counts.shape


In [ ]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)


In [ ]:
from sklearn.decomposition import PCA
#print(X.shape)
pca = PCA(n_components=32)
result_pca = pca.fit_transform(X_train_tfidf.toarray())
print(result_pca)
print(pca)

In [ ]:
traing_transformed = X_train_tfidf

In [ ]:
record_accuracy = True

In [ ]:
fraction = 3400
svd_transformed =traing_transformed
X_test = svd_transformed[fraction:]
Y_test = classes[fraction:]
X_train = svd_transformed[:fraction]
y_train = classes[:fraction]

In [ ]:
from sklearn.linear_model import LogisticRegression
clf_lr = LogisticRegression(C=1,solver='saga',class_weight='balanced' )
clf_lr.fit(X_train, y_train)
print(clf_lr.score(X_train,y_train))
t1 = clf_lr.score(X_test,Y_test)
if record_accuracy:
    v1=t1

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf_rdf = RandomForestClassifier(n_estimators=500,max_depth=6,random_state=40)

clf_rdf.fit(X_train, y_train)
print(clf_rdf.score(X_train, y_train))
t2=clf_rdf.score(X_test,Y_test)
if record_accuracy:
    v2=t2

In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf_nb = MultinomialNB()
alpha=[]
for i in range(1,1000):
    alpha.append(i/1.0)
parameters = {'alpha':alpha}
search = GridSearchCV(clf_nb, parameters, cv=5)
clf_nb = search.fit(X_train, y_train)


print(clf_nb.score(X_train, y_train))
t3=clf_nb.score(X_test,Y_test)
if record_accuracy:
    v3=t3
search.best_params_

In [ ]:
X_submission_counts = count_vect.transform(submission_test_set_formatted)
submission_transormed=tfidf_transformer.transform(X_submission_counts)

predicted_clf_rdf = clf_rdf.predict(submission_transormed)
predicted_clf_lr = clf_lr.predict(submission_transormed)
predicted_clf_nb = clf_nb.predict(submission_transformed)

rs_sub = []
for p in zip(predicted_clf_lr):
    rs_sub.append(int(p))
    
predicted=rs_sub

In [ ]:
import csv
with open('rs.csv', 'w') as csvfile:
    writer = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(['Index','Label'])
    for i in range(len(predicted)):
        writer.writerow([i+1,predicted[i]])